In [3]:
# imports
import pandas as pd
import json
import requests
import folium
print ("libraries imported")

libraries imported


## I CREATED A DATAFRAME OF STATIONS TO WORK WITH

In [4]:
stationsurl = "http://api.citybik.es/v2/networks/broward"
stations_r=requests.get(stationsurl)
stations_r.json=json.loads(stations_r.text)


In [5]:
stations_parsed_data = []
for info in stations_r.json['network']['stations']:
    stations_info_data = {
        "name": info['name'],
        "latitude": info['latitude'],
        "longitude": info['longitude'],
        "free_bikes": info['free_bikes'],
        "empty_slots": info['empty_slots'],
    }
    stations_parsed_data.append(stations_info_data)

In [6]:
stations_df = pd.DataFrame(stations_parsed_data)
stations_df


,name,latitude,longitude,free_bikes,empty_slots
0,Hollywood North Beach,26.03444,-80.11455,1,10
1,Earl Lifshey 3.0,26.16739,-80.10032,10,2
2,Atlantic & Briny,26.23237,-80.08965,6,3
3,Esplanade Park,26.12026,-80.14819,6,5
4,Bayshore & A1A 3.0,26.12807,-80.10368,12,5
5,Holiday Park at Sunrise,26.13707,-80.12981,9,2
6,Broadwalk at Jefferson St.,26.00656,-80.11600,17,0
7,Las Olas Beach Park,26.11837,-80.10492,5,9
8,Las Olas & SE 9th Ave,26.11954,-80.13407,9,2
9,George English Park,26.13813,-80.11570,5,6


In [84]:
# I'm saving the dataframe to CSV for the next section
stations_df.to_csv('stations.csv', index=False)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

# I CREATED A FUNCTION TO ITERATE OVER THE DATAFRAME OF STATIONS

In [7]:
def getNearbyPOI(latitude, longitude):
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
    "radius":"1000",
  	"ll": f"{latitude},{longitude}",
  	"open_now": "true",
  	"sort":"DISTANCE"
    }

    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3KKRisSK1dwYHI3BlvESFIU4LjVkyURq0ipp9jNCHwxc="
    }

    response = requests.request("GET", url, params=params, headers=headers)

    r_json=json.loads(response.text)
    
    return r_json


In [8]:
fspoi = []

for index, station in stations_df.iterrows():
    fspoi.append(getNearbyPOI(station['latitude'],station['longitude']))   


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [10]:
import pandas as pd

fs_parsed_poi_data = []    

for poi in fspoi:
    for business in poi['results']:
            fs_info_data = {
                "name": business['name'],
                "distance": business['distance'],
                "address": business['location']['formatted_address'],
            }
            fs_parsed_poi_data.append(fs_info_data)

stations_list = stations_df['name'].tolist()
    
fs_parsed_poi_data

[{'name': 'Hollywood North Beach Park',
  'distance': 9,
  'address': '3601 N Ocean Dr, Hollywood, FL 33019'},
 {'name': 'Zacks Snacks BBQ',
  'distance': 37,
  'address': '3601 N Ocean Dr, Hollywood, FL 33019'},
 {'name': 'Food King',
  'distance': 414,
  'address': '3111 N Surf Rd, Hollywood, FL 33019'},
 {'name': 'Oceans 13 Sports Bar & Grill',
  'distance': 432,
  'address': '3111 N Surf Rd, Hollywood, FL 33019'},
 {'name': 'Anne Kolb Nature Center',
  'distance': 614,
  'address': '751 Sheridan St, Hollywood, FL 33019'},
 {'name': 'Latitudes Restaurant',
  'distance': 787,
  'address': '2501 N Ocean Dr (at Hollywood Beach Marriott), Hollywood, FL 33019'},
 {'name': 'Walgreens',
  'distance': 84,
  'address': '3101 N Ocean Blvd, Fort Lauderdale, FL 33308'},
 {'name': 'Cafe Martorano',
  'distance': 92,
  'address': '3343 E Oakland Park Blvd (Oakland Park Blvd & A1A), Fort Lauderdale, FL 33308'},
 {'name': 'Scuba School & Dive Center',
  'distance': 101,
  'address': '3329 E Oakland

Put your parsed results into a DataFrame

In [12]:
fs_parsed_poi_data_df = pd.DataFrame(fs_parsed_poi_data)

fs_parsed_poi_data_df

## Sorry, I ran out of time to make it more legible and pretty, but the basic logic is there
## If I had time, I'd run a zip on the stations_list so that all the stations would correspond to their points of interest


,name,distance,address
0,Hollywood North Beach Park,9,"3601 N Ocean Dr, Hollywood, FL 33019"
1,Zacks Snacks BBQ,37,"3601 N Ocean Dr, Hollywood, FL 33019"
2,Food King,414,"3111 N Surf Rd, Hollywood, FL 33019"
3,Oceans 13 Sports Bar & Grill,432,"3111 N Surf Rd, Hollywood, FL 33019"
4,Anne Kolb Nature Center,614,"751 Sheridan St, Hollywood, FL 33019"
...,...,...,...
186,Ocean2000,119,"2000 N Ocean Blvd, Fort Lauderdale, FL 33305"
187,Fort Lauderdale Beach Inn,196,"2197 N Ocean Blvd, Fort Lauderdale, FL 33305"
188,Art of Petals,203,"2200 NE 33rd Ave, Fort Lauderdale, FL 33305"
189,Lu Deaner Park,214,"2125 NE 33rd Ave, Fort Lauderdale, FL 33305"


In [13]:
# I'm saving the dataframe to CSV for the next section
fs_parsed_poi_data_df.to_csv('FourSquarePOI.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [56]:
# imports
import pandas as pd
import json
import requests
print ("libraries imported")


libraries imported


In [57]:
def getNearbyYelp(latitude, longitude):
    
    
    # Define my API Key, My Endpoint, and My Header
    API_KEY = '4RFCgRndWzUphm4vgIsFet58SboTR76bj7eTDXFL5wwhS3_E6RZ8qnye_qVdiaXqgr1FbR1Y9QxzruU6a6AlcbjRGAtsgaDfis7WsRQY9GTRf4c2AVPibpzdt2P4ZHYx'
    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    HEADERS = {'Authorization': 'bearer %s' % API_KEY}
   
    
    # BUSINESS SEARCH PARAMETERS 
    PARAMETERS = {'latitude': f"{latitude}",
              'longitude': f"{longitude}",
              'radius': 1000}   
    
    
    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)   
    
    yelp_data = response.json() 
    
    return yelp_data



In [65]:
yelppoi = []

for index, station in stations_df.iterrows():
    yelppoi.append(getNearbyYelp(station['latitude'],station['longitude']))
    
yelppoi

[{'businesses': [{'id': '3CnH5I51i71TBrZdKLIZtw',
    'alias': 'hollywood-north-beach-park-hollywood',
    'name': 'Hollywood North Beach Park',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ZXl_PlUaLimQq6beZNNC4Q/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/hollywood-north-beach-park-hollywood?adjust_creative=kpzLS5ISuhla30WPzK4cQw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=kpzLS5ISuhla30WPzK4cQw',
    'review_count': 42,
    'categories': [{'alias': 'parks', 'title': 'Parks'},
     {'alias': 'beaches', 'title': 'Beaches'}],
    'rating': 4.5,
    'coordinates': {'latitude': 26.034244, 'longitude': -80.11487},
    'transactions': [],
    'location': {'address1': '3601 N Ocean Dr',
     'address2': '',
     'address3': '',
     'city': 'Hollywood',
     'zip_code': '33019',
     'country': 'US',
     'state': 'FL',
     'display_address': ['3601 N Ocean Dr', 'Hollywood, FL 33019']},
    'phone': '+19543575187',
    'display

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [76]:
import pandas as pd

parsed_yelp_poi_data = []    

for poi in yelppoi:
    for business in poi['businesses']:
            yelp_info_data = {
                "name": business['name'],
                "distance": business['distance'],
                "address": business['location']['address1'],
            }
            parsed_yelp_poi_data.append(yelp_info_data)

parsed_yelp_poi_data

[{'name': 'Hollywood North Beach Park',
  'distance': 38.69347164611035,
  'address': '3601 N Ocean Dr'},
 {'name': 'Oceans 13 Sports Bar & Grill',
  'distance': 451.2825705548206,
  'address': '3111 N Surf Rd'},
 {'name': 'Latitudes Restaurant Hollywood',
  'distance': 794.7312294770691,
  'address': '2501 North Ocean Dr'},
 {'name': 'Anne Kolb Nature Center',
  'distance': 685.5431287866869,
  'address': '751 Sheridan St'},
 {'name': 'Keating Beach',
  'distance': 62.695768582443854,
  'address': '1175-1199 Surf Rd'},
 {'name': 'Hot Dog Harbor of Hollywood',
  'distance': 750.0740223493926,
  'address': '2500 N Surf Rd'},
 {'name': 'Hollywood North Beach Park Green',
  'distance': 653.7322898714801,
  'address': '4400 North Ocean Dr'},
 {'name': 'The Carpenter House',
  'distance': 709.8767020906763,
  'address': '4414 N Surf Rd'},
 {'name': 'Shooters Waterfront',
  'distance': 361.5732535929614,
  'address': '3033 NE 32nd Ave'},
 {'name': 'Greek Islands Taverna',
  'distance': 213.7

Put your parsed results into a DataFrame

In [77]:
parsed_yelp_poi_data_df = pd.DataFrame(parsed_yelp_poi_data)

parsed_yelp_poi_data_df

,name,distance,address
0,Hollywood North Beach Park,38.693472,3601 N Ocean Dr
1,Oceans 13 Sports Bar & Grill,451.282571,3111 N Surf Rd
2,Latitudes Restaurant Hollywood,794.731229,2501 North Ocean Dr
3,Anne Kolb Nature Center,685.543129,751 Sheridan St
4,Keating Beach,62.695769,1175-1199 Surf Rd
...,...,...,...
372,Lu Deaner Park,217.537675,2125 NE 33rd Ave
373,Willingham Park,78.929008,2100 N Atlantic Blvd
374,Loggerhead Park,741.500463,3698 NE 27th St
375,Moreth House,508.069849,3000 NE 19th St


In [80]:
# I'm saving the dataframe to CSV for the next section
parsed_yelp_poi_data_df.to_csv('YelpPOI.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

### Generally yelp seems to provide a higher quality of information considering they give a rating to the businesses involved

Get the top 10 restaurants according to their rating

In [75]:
    # Define my API Key, My Endpoint, and My Header
    API_KEY = '4RFCgRndWzUphm4vgIsFet58SboTR76bj7eTDXFL5wwhS3_E6RZ8qnye_qVdiaXqgr1FbR1Y9QxzruU6a6AlcbjRGAtsgaDfis7WsRQY9GTRf4c2AVPibpzdt2P4ZHYx'
    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    HEADERS = {'Authorization': 'bearer %s' % API_KEY}
   
    
    # BUSINESS SEARCH PARAMETERS 
    PARAMETERS = {'location': "Fort Lauderdale",
              'term': "restaurants",
                  'sort_by': 'rating',
              'limit': 10}
    
    
    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)   
    
    top_10_restaurants = response.json()
    
    top_10_restaurants

{'businesses': [{'id': 'Wp3cWqQ-BbYHaEm6y_DGYw',
   'alias': 'danlu-cafe-fort-lauderdale',
   'name': 'Danlu Cafe',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/qvaj5sg7zIEcc24wzQIkIQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/danlu-cafe-fort-lauderdale?adjust_creative=kpzLS5ISuhla30WPzK4cQw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=kpzLS5ISuhla30WPzK4cQw',
   'review_count': 7,
   'categories': [{'alias': 'bagels', 'title': 'Bagels'},
    {'alias': 'sandwiches', 'title': 'Sandwiches'},
    {'alias': 'cafes', 'title': 'Cafes'}],
   'rating': 5.0,
   'coordinates': {'latitude': 26.1220286, 'longitude': -80.140828},
   'transactions': [],
   'location': {'address1': '200 E Broward Blvd',
    'address2': 'Ste 110',
    'address3': '',
    'city': 'Fort Lauderdale',
    'zip_code': '33301',
    'country': 'US',
    'state': 'FL',
    'display_address': ['200 E Broward Blvd',
     'Ste 110',
     'Fort Lauderdale, FL 33301']},


# I HAD TO MOVE ON DUE TO TIME CONSTRAINTS, BUT IF I HAD TIME I'D PARSE THE DATA AND PUT IT IN A DATAFRAME